# Setup

## Packages

In [1]:
import os
import yaml
import pandas as pd
import dask.dataframe as dd
# import dask

# set up
with open("/home/ubuntu/work/therapeutic_accelerator/config/main.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
with open("../config/keys.yaml", "r") as f:
    keys = yaml.load(f, Loader=yaml.FullLoader)

In [2]:
from sqlalchemy.engine import URL
from sqlalchemy import create_engine, text

url_object = URL.create(
    'postgresql', 
    username='postgres',
    password=keys["postgres"], 
    host=config["database"]["host"],
    database='postgres',
    port=5432
)

# Create engine to connect to database
# psql_string = f'postgresql://postgres:{keys["postgres"]}@{config["database"]["host"]}:5432/postgres'
engine = create_engine(url_object)

## Tokenizers and Models

In [3]:
# import pytorch
max_sequence_length = 1200
embedding_size = 200

from transformers import T5Tokenizer # AutoModel, AutoTokenizer, BertTokenizer,BioGptModel, BioGptConfig, BioGptTokenizer
T5tokens = T5Tokenizer.from_pretrained('t5-base', model_max_length = max_sequence_length)

In [4]:
# bio_bert_model = AutoModel.from_pretrained("gsarti/biobert-nli")
# bio_bert_tokenizer = AutoTokenizer.from_pretrained("gsarti/biobert-nli")
# original_bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# T5Abstract_model = TFT5ForConditionalGeneration.from_pretrained('t5-base')
# biogpttokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
# biogptmodel = BioGptModel.from_pretrained("microsoft/biogpt")

# Pilot Test: Encodings

In [5]:
# # pull all the abstracts
# sql = '''SELECT * FROM abstracts LIMIT 5'''
# with engine.connect() as conn: 
#     results = conn.execute(text(sql))

# # Turn into dataframe    
# abstracts = pd.DataFrame(results.fetchall())

# # Remove empty abstracts
# print("Shape before: ", abstracts.shape)
# abstracts = abstracts.dropna(how = 'all', subset='abstract').reset_index(drop = True)
# print("Shape after: ", abstracts.shape)

# # Distribution of abstract lengths
# ab_lens = abstracts.abstract.apply(lambda x: len(x.split()))

# # Lengths of Abstracts
# display(ab_lens.describe())

# from seaborn import displot
# displot(ab_lens)

In [6]:
# # create new columns for DB table
# abstracts[['input_ids', 'attention_mask']] = abstracts.abstract.apply(T5tokens).apply(pd.Series)

# # Cleanup
# abstracts.drop(['index', 'id'], axis = 1, inplace = True)

# # QA check
# abstracts

# Create Encodings using Dask

In [7]:
ddf = dd.read_sql_table('abstracts', 
                        con = f'postgresql://postgres:{keys["postgres"]}@{config["database"]["host"]}:5432/postgres',
                        index_col = 'id',
                        head_rows = 5,
                        npartitions = 700)

# Remove unnecessary columns
ddf = ddf.drop(columns = ['index'])

# Remove empty abstract rows
ddf = ddf.dropna(how = 'all', subset='abstract').reset_index(drop = True)

In [8]:
import numpy as np

In [9]:
# create new columns for DB table
ddf2 = ddf.abstract.apply(T5tokens, meta=('abstract', 'string')).apply(pd.Series, meta=({'input_ids':'object', 'attention_mask':'object'}))

# concatenate two dataframes
ddf = dd.concat([ddf, ddf2], axis = 1) # create divisions in ddfs? 
# QA check
# results

/home/ubuntu/anaconda3/lib/python3.10/site-packages/dask/dataframe/multi.py:1291: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


In [10]:
ddf

,paperId,corpusId,abstract,input_ids,attention_mask
npartitions=700,,,,,
,object,object,object,object,object
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [11]:
# Rename columns for easier reading later
ddf.columns 

Index(['paperId', 'corpusId', 'abstract', 'input_ids', 'attention_mask'], dtype='object')

In [ ]:
# ddf3.dask

In [ ]:
# Save backup
# name_function = lambda x: f"abstracts-{x}.parquet"
# ddf3.to_parquet('/home/ubuntu/work/backup/', name_function = name_function)

# Upload to Postgresql DB

In [ ]:
# sql = text(''' 
#     SELECT EXISTS (
#         SELECT FROM information_schema.tables 
#         WHERE    table_name   = 'abstracts'
#     );
# ''')

# with engine.connect() as conn: 
#     conn.execute(sql)

Create array columns to store encoding and mask

In [12]:
sql = text(''' 
    DROP TABLE IF EXISTS abstracts_encodings;
''')

with engine.connect() as conn: 
    query = conn.execute(sql)

In [13]:
table_name = 'abstracts_encodings'

In [14]:
# create table
# Create Table in DB first before uploading
from sqlalchemy import MetaData, Table, Column, Integer, String, ARRAY

metadata_obj = MetaData()

# Create abstracts metadata
abstracts = Table(
    table_name,
    metadata_obj,
    Column("paperId", String, nullable = True),
    Column("corpusId", String, nullable=True),
    Column("abstract", String, nullable = True),
    Column("input_ds", ARRAY(Integer), nullable=True),
    Column("attention_mask", ARRAY(Integer), nullable=True),
)

metadata_obj.create_all(engine)

In [15]:
# Upload dask dataframe to psql
ddf = ddf.to_sql(name = table_name, uri = str(url_object), if_exists = 'replace', index = False, chunksize = 10000, method = 'multi')

Token indices sequence length is longer than the specified maximum sequence length for this model (1482 > 1200). Running this sequence through the model will result in indexing errors


ProgrammingError: (psycopg2.errors.UndefinedTable) relation "abstracts_encodings" does not exist
LINE 1: INSERT INTO abstracts_encodings ("paperId", "corpusId", abst...
                    ^

[SQL: INSERT INTO abstracts_encodings ("paperId", "corpusId", abstract, input_ids, attention_mask) VALUES (%(paperId_m0)s, %(corpusId_m0)s, %(abstract_m0)s, %(input_ids_m0)s, %(attention_mask_m0)s), (%(paperId_m1)s, %(corpusId_m1)s, %(abstract_m1)s, %(input_ids_m1)s, %(attention_mask_m1)s), (%(paperId_m2)s, %(corpusId_m2)s, %(abstract_m2)s, %(input_ids_m2)s, %(attention_mask_m2)s), (%(paperId_m3)s, %(corpusId_m3)s, %(abstract_m3)s, %(input_ids_m3)s, %(attention_mask_m3)s), (%(paperId_m4)s, %(corpusId_m4)s, %(abstract_m4)s, %(input_ids_m4)s, %(attention_mask_m4)s), (%(paperId_m5)s, %(corpusId_m5)s, %(abstract_m5)s, %(input_ids_m5)s, %(attention_mask_m5)s), (%(paperId_m6)s, %(corpusId_m6)s, %(abstract_m6)s, %(input_ids_m6)s, %(attention_mask_m6)s), (%(paperId_m7)s, %(corpusId_m7)s, %(abstract_m7)s, 

In [ ]:
# Check if it worked
import pandas as pd

sql = text(f''' 
    SELECT * FROM {table_name} LIMIT 5;
''')

with engine.connect() as conn: 
    query = conn.execute(sql)

test = pd.DataFrame(query.fetchall())
test.head()